#https://python.langchain.com/docs/integrations/document_loaders/pypdfloader/

In [ ]:
import sys
print(sys.executable)

Import all envirnments from .env file

In [ ]:
import os,certifi
from langchain_groq import chat_models
from dotenv import load_dotenv
load_dotenv()

#  os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
#os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["SSL_CERT_FILE"] = certifi.where()
os.environ["REQUESTS_CA_BUNDLE"] = certifi.where()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
# GROQ_API_KEY
LANGCHAIN_API_KEY  = os.getenv("LANGCHAIN_API_KEY")
#LANGCHAIN_API_KEY
FILE_PATH = os.environ.get("FILE_PATH")
# FILE_PATH

In [ ]:
from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM, OllamaEmbeddings

llm = Ollama(model="gemma3:4b")   # This one generates text


Initialization

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

In [ ]:
file_path = FILE_PATH
loader = PyPDFLoader(file_path)

In [ ]:
loader

Load

In [ ]:
docs = loader.load()
print(docs[0].page_content)      # The text from the first page
print(docs[0].metadata)    

In [ ]:
type(docs)

In [ ]:
print(len(docs))
# print.pp(docs[0].metadata)

 2. Initialize the recursive text splitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # max characters per chunk
    chunk_overlap=50,  # overlap to preserve context
)

In [ ]:
text_splitter

3. Recursively split the documents

In [ ]:
final_document = text_splitter.split_documents(docs)

In [ ]:
final_document 

In [ ]:
type(final_document)

In [ ]:
print(f"Total chunks: {len(final_document)}")
print(final_document[0])
print(final_document[1])

Embeddings

In [ ]:
from langchain_ollama import OllamaEmbeddings

In [ ]:

embeddings_model = OllamaEmbeddings(
    # model="gemma3:4b"
    model="nomic-embed-text"
)

In [ ]:
embeddings_model

In [ ]:
from langchain.vectorstores import Chroma
from tqdm import tqdm
import logging

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
# vectorstore = FAISS.load_local(
#     "faiss_index",
#     embeddings_model,
#     allow_dangerous_deserialization=True
# )
vectorstore = FAISS.from_documents(final_document, embeddings_model)


In [ ]:
vectorstore 

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
# # Create retriever from vectorstore to take a data from vectorstoredb
# vectorstore = FAISS.load_local("faiss_index", embeddings_model)
# retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # type: ignore

In [ ]:
prompt = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"   # can also try "map_reduce" or "refine"
)
prompt

In [ ]:
#query = "What was Shivaji's role in promoting the navy?"
#query = "Who is Shivaji?"
#query ="Shivaji's birthday"
query = "who is Shivaji great grandfather?"


In [ ]:
answer = prompt.invoke({"query": query})
print(answer['result'])


#### Rough work

In [ ]:
# vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings_model)
# for i, chunk in enumerate(tqdm(final_document, desc="Processing chunks"), 1):
#     vectorstore.add_documents([chunk], embedding=embeddings_model)
#     logging.info(f"Stored {len(final_document)} chunks in the vectorstore.")

In [ ]:
#query = "What was Shivaji's role in promoting the navy?"
#query = "Who is Shivaji?"
query ="Shivaji's birthday"
results = vectorstore.similarity_search(query)  # k = number of top results
print(results[0].page_content)
# for i, doc in enumerate(results, 1):
#     print(f"Result {i}:")
#     print(doc.page_content)
#     print("-" * 40)


In [ ]:
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma3:4b",groq_api_key= GROQ_API_KEY) 
model